In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [3]:
data=pd.read_csv("/content/dataset.csv",index_col=0)

In [4]:
data['Timestamp'] = pd.to_datetime(data['LastUpdatedDate'] + ' ' + data['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')
data = data.sort_values('Timestamp').reset_index(drop=True)

In [5]:
data["VehicleType"][data["VehicleType"]=="cycle"]=1
data["VehicleType"][data["VehicleType"]=="bike"]=2
data["VehicleType"][data["VehicleType"]=="car"]=3
data["VehicleType"][data["VehicleType"]=="truck"]=4
data["TrafficConditionNearby"][data["TrafficConditionNearby"]=="low"]=1
data["TrafficConditionNearby"][data["TrafficConditionNearby"]=="average"]=2
data["TrafficConditionNearby"][data["TrafficConditionNearby"]=="high"]=3
data["TrafficConditionNearby"]=(data["TrafficConditionNearby"]).astype(int)
data["VehicleType"]=(data["VehicleType"]).astype(int)
data["IsSpecialDay"]=(data["IsSpecialDay"]).astype(int)
data["Occupancy"]=(data["Occupancy"]).astype(int)
data["Capacity"]=(data["Capacity"]).astype(int)
data["Occupancy_rate"]=data["Occupancy"]/data["Capacity"]

/tmp/ipython-input-5-2597378481.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data["VehicleType"][data["VehicleType"]=="cycle"]=1
/tmp/ipython-input-5-2597378481.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [6]:
class ParkingSchema2(pw.Schema):
  Timestamp : str
  VehicleType : int
  TrafficConditionNearby : int
  IsSpecialDay : int
  Occupancy_rate : float
  QueueLength : int

In [7]:
data[["Timestamp","VehicleType","TrafficConditionNearby","IsSpecialDay","Occupancy_rate","QueueLength"]].to_csv("parking_stream2.csv",index=False)

In [8]:
data_stream2=pw.demo.replay_csv("/content/parking_stream2.csv",schema=ParkingSchema2,input_rate=1800)

In [9]:
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time2 = data_stream2.with_columns(
    t = data_stream2.Timestamp.dt.strptime(fmt),
    day = data_stream2.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [32]:
import datetime
delta_window2 = (
    data_with_time2.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(duration=datetime.timedelta(minutes=30)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        vehtype=pw.reducers.avg(pw.this.VehicleType),
        traffic=pw.reducers.max(pw.this.TrafficConditionNearby),
        special=pw.reducers.max(pw.this.IsSpecialDay),
        occ_rate_avg=pw.reducers.avg(pw.this.Occupancy_rate),
        queue=pw.reducers.max(pw.this.QueueLength),
    )
    .with_columns(
        price=pw.if_else(pw.this.queue>13,10*(1+4*(((pw.this.vehtype)/4)-(pw.this.traffic/3)**2+2*(pw.this.special)+(pw.this.occ_rate_avg)+((pw.this.queue)/15**0.5))/25),
              pw.if_else(pw.this.queue>10 ,10*(1+4*(((pw.this.vehtype)/4)-(pw.this.traffic/3)**2+2*(pw.this.special)+(pw.this.occ_rate_avg)+((pw.this.queue)/15**0.7))/25),
              pw.if_else(pw.this.queue>5 ,10*(1+3*(((pw.this.vehtype)/4)-(pw.this.traffic/3)**2+2*(pw.this.special)+(pw.this.occ_rate_avg)+((pw.this.queue)/15))/25),
              pw.if_else(pw.this.queue>3 ,10*(1+((pw.this.vehtype/4)+pw.this.occ_rate_avg+pw.this.special)/6),10)))),

    )

)

In [33]:
pn.extension()
def price_plotter2(source):

    fig = bokeh.plotting.figure(
        height=400,
        width=1000,
        title="Pathway: Daily Parking Price_Model2",
        x_axis_type="datetime",
    )

    fig.line("t", "price", source=source, line_width=2, color="green")


    fig.circle("t", "price", source=source, size=6, color="navy")

    return fig
viz = delta_window2.plot(price_plotter2, sorting_col="t")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [34]:
%%capture --no-display
pw.run()

Output()